In [1]:
from abc import ABC, abstractmethod
import json
import openai
from dotenv import load_dotenv, dotenv_values

# データをベクトル化するモジュールのインターフェース
class Embedder(ABC):

    @abstractmethod
    def embed(self, texts: list[str]) -> list[list[float]]:
        raise NotImplementedError

    @abstractmethod
    def save(self, texts: list[str], filename: str) -> bool:
        raise NotImplementedError


# Embedderインターフェースの実装
class OpenAIEmbedder(Embedder):
    def __init__(self, api_key: str):
        openai.api_key = api_key

    def embed(self, texts: list[str]) -> list[list[float]]:
        # openai 1.10.0 で動作確認
        response = openai.embeddings.create(input=texts, model="text-embedding-3-small")
        # レスポンスからベクトルを抽出
        return [data.embedding for data in response.data]

    def save(self, texts: list[str], filename: str) -> bool:
        vectors = self.embed(texts)
        data_to_save = [
            {"id": idx, "text": text, "vector": vector}
            for idx, (text, vector) in enumerate(zip(texts, vectors))
        ]
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data_to_save, f, ensure_ascii=False, indent=4)
        print(f"{filename} に保存されました。")
        return True


if __name__ == "__main__":
    import os
    #from os.path import join, dirname, abspath

    texts = [
        "佐藤一郎は、東京生まれの35歳のプログラマーです。趣味は写真撮影とハイキング。新しい技術を学ぶことに情熱を注いでいます。",
        "鈴木花子は、北海道出身の28歳のイラストレーターです。猫を二匹飼っており、自然と動物を愛する心優しい人物です。",
        "田中健二は、大阪で小さなカフェを経営する45歳の起業家です。コーヒーに対する深い知識と情熱を持ち、地域社会に貢献しています。",
        "山本美咲は、福岡県出身の22歳の大学生です。法律を専攻しており、将来は人権に関わる仕事に就きたいと考えています。",
        "伊藤高志は、長野県の山の中で育った30歳の写真家です。自然の美しさを捉えることに特化し、国内外で展示会を開催しています。",
        "小林由紀子は、沖縄県出身の40歳の小学校教師です。子どもたちに芸術と文化の大切さを教えることに生きがいを感じています。",
    ]

    # OpenAI APIキーを事前に環境変数にセットしてください。
    # specify the name of the .env file name 
    env_name = "example.env" # following example.env template change to your own .env file name
    config = dotenv_values(os.path.join(os.getcwd(), env_name), encoding='UTF-16LE')
    api_key = config['openai_api_key']

    if api_key is None:
        raise ValueError("APIキーがセットされていません。")

    embedder = OpenAIEmbedder(api_key)
    embedder.save(texts, "sample_data.json")

sample_data.json に保存されました。


In [5]:
from abc import ABC, abstractmethod
import json
import numpy as np


class NearestNeighborsFinder(ABC):
    @abstractmethod
    def find_nearest(self, vector: list[float], topk: int = 3) -> list[dict]:
        pass


class CosineNearestNeighborsFinder(NearestNeighborsFinder):
    def __init__(self, data_file: str):
        self.data = self._load_data(data_file)

    def _load_data(self, data_file: str) -> list[dict]:
        with open(data_file, "r", encoding="utf-8") as f:
            return json.load(f)

    def _cosine_similarity(self, vec1: list[float], vec2: list[float]) -> float:
        vec1 = np.array(vec1)
        vec2 = np.array(vec2)
        # openAI embeddingのベクトルを対象にする場合は正規化されているため、np.dot(vec1, vec2) だけでも良い
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

    def find_nearest(self, vector: list[float], topk: int = 1) -> list[dict]:
        similarities = [
            (idx, self._cosine_similarity(vector, item["vector"]))
            for idx, item in enumerate(self.data)
        ]
        # 類似度が高い順にソート
        sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
        # Top-Kの結果を返す
        return [self.data[idx] for idx, _ in sorted_similarities[:topk]]

In [6]:
from abc import ABC, abstractmethod
from openai import OpenAI


class ChatBot(ABC):
    @abstractmethod
    def generate_response(self, user_query: str, refs: list[str]) -> str:
        pass


class GPTBasedChatBot(ChatBot):
    def __init__(self):
        self.client = OpenAI()

    def generate_response(self, user_query: str, refs: list[str]) -> str:

        # GPTによる応答を生成
        context = "\n".join(refs) + "\n"

        prompt = f"以下の情報に基づいてユーザーの質問に答えてください:\n\n{context}\n\n質問: {user_query}\n答え:"
        print("#" * 30)
        print("#" * 30)
        print(f"\nprompt:\n {prompt}\n")
        print("#" * 30)
        print("#" * 30)

        completion = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
        )
        return completion.choices[0].message.content

In [7]:
import os
from embedder import OpenAIEmbedder
from searcher import CosineNearestNeighborsFinder
from chatBot import GPTBasedChatBot

# OpenAI APIキーを事前に環境変数にセットしてください。
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    raise ValueError("APIキーがセットされていません。")


def main():

    embedder = OpenAIEmbedder(api_key)
    searcher = CosineNearestNeighborsFinder("sample_data.json")

    user_query: str = "アートを教えられる先生を探しています"

    user_query_vector: list[float] = embedder.embed([user_query])[0]
    search_results: list[dict] = searcher.find_nearest(user_query_vector, topk=2)
    chat_bot = GPTBasedChatBot()
    response: str = chat_bot.generate_response(
        user_query, [search_result["text"] for search_result in search_results]
    )

    print("*" * 30)
    print("*" * 30)
    print("【AIの返答】")
    print(response)


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'embedder'